# SBOM Detection Strategy: CI Workflow Analysis

## Objective
We've found only 8 SBOMs in release artifacts, but many repos likely generate SBOMs in CI that aren't published. This notebook uses FTS and JSON processing to detect SBOM generation in GitHub Actions workflows.

## Strategy
1. **Current State**: Analyze the 8 known SBOMs
2. **FTS Tool Detection**: Search workflows for SBOM generation tools (syft, trivy, cdxgen, etc.)
3. **Pattern Extraction**: Identify where and how SBOMs are generated
4. **Gap Analysis**: Find repos generating SBOMs but not publishing them
5. **Interactive Exploration**: Use pygwalker for ad-hoc analysis

## Setup & Configuration

In [1]:
import duckdb
import pandas as pd
import numpy as np
import altair as alt
import pygwalker as pyg
from pathlib import Path
from rich import print as rprint
from rich.console import Console
from rich.table import Table
import warnings

warnings.filterwarnings('ignore')
console = Console()

# Database path
DB_PATH = 'output/cncf-full-landscape-2025-10-21T00-24-35/GetRepoDataExtendedInfo/database.db'

# Connect to DuckDB
conn = duckdb.connect(DB_PATH, read_only=True)

print("✅ Setup complete!")
print(f"📊 Database: {DB_PATH}")

IOException: IO Error: Cannot open database "/Users/matt/gh/me/github-repo-supply-chain-datacollector/notebooks/output/cncf-full-landscape-2025-10-21T00-24-35/GetRepoDataExtendedInfo/database.db" in read-only mode: database does not exist

## 1. Current State: Known SBOM Artifacts

Let's start by understanding what we already know about the 8 SBOMs found in release artifacts.

In [ ]:
# Query current SBOM artifacts
query = """
SELECT 
    nameWithOwner,
    asset_name,
    sbom_format,
    download_url
FROM agg_artifact_patterns
WHERE is_sbom = true
ORDER BY nameWithOwner, asset_name;
"""

sbom_artifacts = conn.execute(query).df()

print(f"\n📦 Found {len(sbom_artifacts)} SBOM artifacts in releases:\n")
display(sbom_artifacts)  # Show all rows, no pagination

# Altair visualization: SBOM format distribution
format_summary = sbom_artifacts['sbom_format'].value_counts().reset_index()
format_summary.columns = ['sbom_format', 'count']
print("\n📊 SBOM Format Distribution:")
display(format_summary)

import altair as alt
chart = alt.Chart(format_summary).mark_bar().encode(
    x=alt.X('sbom_format:N', title='SBOM Format'),
    y=alt.Y('count:Q', title='Count'),
    color='sbom_format:N',
    tooltip=['sbom_format', 'count']
).properties(title='SBOM Format Distribution')
chart.display()

## 2. Tool Usage Summary

Check what the existing analysis tells us about SBOM tool usage.

In [ ]:
# Query SBOM tool usage from existing analysis
query = """
SELECT 
    COUNT(DISTINCT repository_id) as total_repos,
    SUM(CASE WHEN uses_syft THEN 1 ELSE 0 END) as repos_using_syft,
    SUM(CASE WHEN uses_trivy THEN 1 ELSE 0 END) as repos_using_trivy,
    SUM(CASE WHEN uses_cdxgen THEN 1 ELSE 0 END) as repos_using_cdxgen,
    SUM(CASE WHEN has_sbom_artifact THEN 1 ELSE 0 END) as repos_with_sbom_artifacts,
    SUM(CASE WHEN uses_sbom_generator THEN 1 ELSE 0 END) as repos_using_sbom_generators
FROM agg_repo_summary;
"""

tool_summary = conn.execute(query).df()
print("\n🔧 SBOM Tool Usage Summary:\n")
display(tool_summary)  # Show all rows, no pagination

# Calculate the gap
gap = tool_summary['repos_using_sbom_generators'].iloc[0] - tool_summary['repos_with_sbom_artifacts'].iloc[0]
print(f"\n⚠️  GAP: {gap} repos use SBOM generators but don't publish SBOM artifacts!")

## 3. FTS Search: SBOM Keywords in Workflows

Use Full-Text Search to find workflows mentioning SBOM-related keywords.

In [ ]:
# FTS search for SBOM keywords in workflow content
query = """
WITH sbom_workflows AS (
    SELECT 
        w.id as workflow_id,
        w.repository_id,
        w.filename,
        r.nameWithOwner,
        r.description,
        -- Use FTS to find SBOM mentions
        fts_main_base_workflows.match_bm25(w.id, 'sbom OR spdx OR cyclonedx', fields := 'content') as sbom_score,
        fts_main_base_workflows.match_bm25(w.id, 'syft OR trivy OR cdxgen OR anchore', fields := 'content') as tool_score
    FROM base_workflows w
    JOIN base_repositories r ON w.repository_id = r.id
    WHERE fts_main_base_workflows.match_bm25(w.id, 'sbom OR spdx OR cyclonedx OR syft OR trivy OR cdxgen', fields := 'content') IS NOT NULL
)
SELECT 
    nameWithOwner,
    filename,
    sbom_score,
    tool_score,
    description
FROM sbom_workflows
ORDER BY sbom_score DESC NULLS LAST, tool_score DESC NULLS LAST
LIMIT 50;
"""

sbom_workflows = conn.execute(query).df()

print(f"\n🔍 Found {len(sbom_workflows)} workflows with SBOM keywords:\n")
display(sbom_workflows.head(20))

# Count unique repos
unique_repos = sbom_workflows['nameWithOwner'].nunique()
print(f"\n📊 {unique_repos} unique repositories have workflows mentioning SBOM keywords")

## 4. Deep Dive: Individual SBOM Tools

Search for specific SBOM generation tools one by one.

In [ ]:
# Define SBOM tools to search for
sbom_tools = {
    'syft': 'anchore/sbom-action OR syft',
    'trivy': 'aquasecurity/trivy-action OR trivy',
    'cdxgen': 'cdxgen OR cyclonedx-cli',
    'spdx-sbom-generator': 'spdx-sbom-generator',
    'sbom-tool': 'microsoft/sbom-tool OR sbom-tool',
    'grype': 'anchore/scan-action OR grype',
    'bom': 'kubernetes-sigs/bom',
}

results = {}

for tool_name, search_pattern in sbom_tools.items():
    query = f"""
    SELECT 
        COUNT(DISTINCT w.repository_id) as repo_count,
        COUNT(w.id) as workflow_count
    FROM base_workflows w
    WHERE fts_main_base_workflows.match_bm25(w.id, '{search_pattern}', fields := 'content') IS NOT NULL;
    """
    
    result = conn.execute(query).df()
    results[tool_name] = {
        'repos': result['repo_count'].iloc[0],
        'workflows': result['workflow_count'].iloc[0]
    }

# Convert to DataFrame
tool_df = pd.DataFrame(results).T.reset_index()
tool_df.columns = ['tool', 'repos', 'workflows']
tool_df = tool_df.sort_values('repos', ascending=False)

print("\n🔧 SBOM Tool Detection in CI Workflows:\n")
display(tool_df)  # Show all rows, no pagination

# Altair horizontal bar chart
import altair as alt
chart = alt.Chart(tool_df).mark_bar().encode(
    x=alt.X('repos:Q', title='Number of Repositories'),
    y=alt.Y('tool:N', sort='-x', title='Tool'),
    color='tool:N',
    tooltip=['tool', 'repos', 'workflows']
).properties(title='SBOM Tools Detected in CI Workflows', height=300)
chart.display()

## 5. Artifact Upload Pattern Detection

Find workflows that generate SBOMs and check if they upload artifacts or attach to releases.

In [ ]:
# Find workflows with SBOM generation AND artifact upload patterns
query = """
WITH sbom_gen_workflows AS (
    SELECT 
        w.id as workflow_id,
        w.repository_id,
        w.filename,
        r.nameWithOwner,
        w.content,
        -- Check for SBOM generation
        fts_main_base_workflows.match_bm25(w.id, 'syft OR trivy OR cdxgen OR sbom-tool', fields := 'content') IS NOT NULL as generates_sbom,
        -- Check for artifact upload
        fts_main_base_workflows.match_bm25(w.id, 'upload-artifact OR upload-release-asset OR softprops/action-gh-release', fields := 'content') IS NOT NULL as uploads_artifact,
        -- Check for GitHub release
        fts_main_base_workflows.match_bm25(w.id, 'create-release OR gh release OR github-release', fields := 'content') IS NOT NULL as creates_release
    FROM base_workflows w
    JOIN base_repositories r ON w.repository_id = r.id
    WHERE fts_main_base_workflows.match_bm25(w.id, 'syft OR trivy OR cdxgen OR sbom-tool', fields := 'content') IS NOT NULL
)
SELECT 
    nameWithOwner,
    filename,
    generates_sbom,
    uploads_artifact,
    creates_release,
    CASE 
        WHEN uploads_artifact OR creates_release THEN '✅ Publishes'
        ELSE '❌ No Publishing'
    END as publishing_status
FROM sbom_gen_workflows
ORDER BY uploads_artifact DESC, creates_release DESC, nameWithOwner;
"""

upload_patterns = conn.execute(query).df()

print(f"\n📤 SBOM Generation and Publishing Patterns:\n")
display(upload_patterns)  # Show all rows, no pagination

# Summary
publishes = upload_patterns['publishing_status'].value_counts().reset_index()
publishes.columns = ['publishing_status', 'count']
print("\n📊 Publishing Summary:")
display(publishes)

import altair as alt
publish_chart = alt.Chart(publishes).mark_bar().encode(
    x=alt.X('publishing_status:N', title='Publishing Status'),
    y=alt.Y('count:Q', title='Count'),
    color='publishing_status:N',
    tooltip=['publishing_status', 'count']
).properties(title='SBOM Generation and Publishing Patterns')
publish_chart.display()

## 6. The Gap: Repos Generating But Not Publishing

Identify repositories that generate SBOMs in CI but don't publish them as release artifacts.

In [ ]:
# Find the gap: repos with SBOM generation in CI but no SBOM artifacts
query = """
WITH repos_with_sbom_ci AS (
    SELECT DISTINCT
        w.repository_id,
        r.nameWithOwner,
        r.description,
        r.url
    FROM base_workflows w
    JOIN base_repositories r ON w.repository_id = r.id
    WHERE fts_main_base_workflows.match_bm25(w.id, 'syft OR trivy OR cdxgen OR sbom-tool OR spdx-sbom-generator', fields := 'content') IS NOT NULL
),
repos_with_sbom_artifacts AS (
    SELECT DISTINCT repository_id
    FROM agg_artifact_patterns
    WHERE is_sbom = true
)
SELECT 
    ci.nameWithOwner,
    ci.description,
    ci.url,
    CASE WHEN art.repository_id IS NOT NULL THEN '✅ Has Artifacts' ELSE '❌ Missing Artifacts' END as artifact_status
FROM repos_with_sbom_ci ci
LEFT JOIN repos_with_sbom_artifacts art ON ci.repository_id = art.repository_id
ORDER BY artifact_status, ci.nameWithOwner;
"""

gap_analysis = conn.execute(query).df()

print(f"\n⚠️  GAP ANALYSIS: SBOM Generation vs. Publication\n")
display(gap_analysis.head(50))

# Summary
gap_summary = gap_analysis['artifact_status'].value_counts()
print("\n📊 Gap Summary:")
print(gap_summary)

missing = gap_summary.get('❌ Missing Artifacts', 0)
has_artifacts = gap_summary.get('✅ Has Artifacts', 0)
print(f"\n🎯 {missing} repos generate SBOMs in CI but don't publish them!")
print(f"✅ {has_artifacts} repos both generate and publish SBOMs")

## 7. Detailed Workflow Content Analysis

Let's examine actual workflow content for a few repos to understand SBOM generation patterns.

In [ ]:
# Get sample workflow content for detailed analysis
query = """
SELECT 
    r.nameWithOwner,
    w.filename,
    w.content
FROM base_workflows w
JOIN base_repositories r ON w.repository_id = r.id
WHERE fts_main_base_workflows.match_bm25(w.id, 'syft OR anchore/sbom-action', fields := 'content') IS NOT NULL
LIMIT 3;
"""

sample_workflows = conn.execute(query).df()

print("\n📋 Sample Workflow Content (First 3):\n")

for idx, row in sample_workflows.iterrows():
    print(f"\n{'='*80}")
    print(f"Repository: {row['nameWithOwner']}")
    print(f"Workflow: {row['filename']}")
    print(f"{'='*80}\n")
    
    # Show first 1500 characters of content
    content = row['content']
    if len(content) > 1500:
        print(content[:1500] + "\n\n... (truncated) ...")
    else:
        print(content)
    print("\n")

## 8. Cross-Reference with Project Metadata

Check if CNCF maturity level correlates with SBOM adoption.

In [ ]:
# Analyze SBOM adoption by CNCF maturity level
query = """
SELECT 
    maturity,
    COUNT(*) as project_count,
    SUM(repos_with_sbom) as projects_with_sbom,
    AVG(sbom_adoption_rate) as avg_adoption_rate,
    SUM(repos_using_syft) as projects_using_syft,
    SUM(repos_using_trivy) as projects_using_trivy,
    SUM(repos_using_cosign) as projects_using_cosign
FROM agg_cncf_project_summary
WHERE maturity IS NOT NULL
GROUP BY maturity
ORDER BY 
    CASE maturity 
        WHEN 'graduated' THEN 1 
        WHEN 'incubating' THEN 2 
        WHEN 'sandbox' THEN 3 
    END;
"""

maturity_analysis = conn.execute(query).df()

print("\n📊 SBOM Adoption by CNCF Maturity Level:\n")
display(maturity_analysis)  # Show all rows, no pagination

import altair as alt
# Chart 1: Adoption rate by maturity
adoption_chart = alt.Chart(maturity_analysis).mark_bar().encode(
    x=alt.X('maturity:N', title='Maturity Level', sort=['graduated', 'incubating', 'sandbox']),
    y=alt.Y('avg_adoption_rate:Q', title='Average SBOM Adoption Rate'),
    color='maturity:N',
    tooltip=['maturity', 'avg_adoption_rate']
).properties(title='SBOM Adoption by Maturity')
adoption_chart.display()

# Chart 2: Tool usage by maturity
tool_cols = ['projects_using_syft', 'projects_using_trivy', 'projects_using_cosign']
tool_usage = maturity_analysis.melt(id_vars=['maturity'], value_vars=tool_cols, var_name='tool', value_name='count')
tool_chart = alt.Chart(tool_usage).mark_bar().encode(
    x=alt.X('maturity:N', title='Maturity Level', sort=['graduated', 'incubating', 'sandbox']),
    y=alt.Y('count:Q', title='Number of Projects'),
    color='tool:N',
    tooltip=['maturity', 'tool', 'count']
).properties(title='Tool Usage by Maturity', height=300)
tool_chart.display()

## 9. Export Gap Analysis for Further Action

Create a CSV of repos that should be publishing SBOMs.

In [ ]:
# Export repos with SBOM generation but no publication
missing_sboms = gap_analysis[gap_analysis['artifact_status'] == '❌ Missing Artifacts']

output_path = 'sbom-publication-gap.csv'
missing_sboms.to_csv(output_path, index=False)

print(f"\n✅ Exported {len(missing_sboms)} repos to {output_path}")
print("\nThese repos generate SBOMs in CI but don't publish them as release artifacts.")
print("Consider reaching out to these projects to encourage SBOM publication!")

## 10. Interactive Exploration with PyGWalker

Use PyGWalker for interactive, visual exploration of the data.

In [ ]:
# Load comprehensive dataset for interactive exploration
query = """
SELECT 
    r.nameWithOwner,
    r.description,
    rs.total_releases,
    rs.total_assets,
    rs.total_workflows,
    rs.has_sbom_artifact,
    rs.sbom_artifact_count,
    rs.uses_syft,
    rs.uses_trivy,
    rs.uses_cdxgen,
    rs.uses_cosign,
    rs.uses_sigstore,
    rs.uses_codeql,
    rs.uses_sbom_generator,
    rs.uses_signer,
    rs.uses_vulnerability_scanner,
    cp.maturity,
    cp.category,
    cp.subcategory
FROM agg_repo_summary rs
JOIN base_repositories r ON rs.repository_id = r.id
LEFT JOIN base_cncf_project_repos cpr ON r.nameWithOwner = cpr.owner || '/' || cpr.name
LEFT JOIN base_cncf_projects cp ON cpr.project_name = cp.project_name;
"""

exploration_df = conn.execute(query).df()

print(f"\n🎨 Loaded {len(exploration_df)} repositories for interactive exploration")
print("\nUsing PyGWalker - this will open an interactive visualization tool below.")
print("You can drag and drop fields to create custom charts and explore patterns!\n")

# Launch PyGWalker
pyg.walk(exploration_df, env='Jupyter')

## 11. Advanced: Workflow Step Extraction (Experimental)

Attempt to extract structured information about SBOM generation steps.

In [ ]:
# Extract lines mentioning SBOM tools to understand common patterns
query = """
SELECT 
    r.nameWithOwner,
    w.filename,
    -- Extract lines with SBOM tools (simplified approach)
    CASE 
        WHEN w.content LIKE '%anchore/sbom-action%' THEN 'anchore/sbom-action'
        WHEN w.content LIKE '%syft%' THEN 'syft'
        WHEN w.content LIKE '%aquasecurity/trivy-action%' THEN 'aquasecurity/trivy-action'
        WHEN w.content LIKE '%trivy%' THEN 'trivy'
        WHEN w.content LIKE '%cdxgen%' THEN 'cdxgen'
        WHEN w.content LIKE '%spdx-sbom-generator%' THEN 'spdx-sbom-generator'
        WHEN w.content LIKE '%microsoft/sbom-tool%' THEN 'microsoft/sbom-tool'
        ELSE 'other'
    END as detected_tool,
    -- Check for artifact upload
    w.content LIKE '%upload-artifact%' as has_upload_artifact,
    w.content LIKE '%upload-release-asset%' as has_upload_release,
    w.content LIKE '%softprops/action-gh-release%' as has_gh_release_action
FROM base_workflows w
JOIN base_repositories r ON w.repository_id = r.id
WHERE fts_main_base_workflows.match_bm25(w.id, 'syft OR trivy OR cdxgen OR sbom-tool', fields := 'content') IS NOT NULL
LIMIT 100;
"""

step_patterns = conn.execute(query).df()

print("\n🔍 Workflow Step Patterns:\n")
display(step_patterns)  # Show all rows, no pagination

# Analyze patterns
tool_upload_matrix = step_patterns.groupby('detected_tool')[[
    'has_upload_artifact', 'has_upload_release', 'has_gh_release_action'
]].sum().reset_index()

print("\n📊 Tool vs. Upload Method Matrix:")
display(tool_upload_matrix)

# Altair stacked bar chart for upload methods
import altair as alt
upload_chart = alt.Chart(tool_upload_matrix.melt(id_vars=['detected_tool'], var_name='upload_method', value_name='count')).mark_bar().encode(
    x=alt.X('detected_tool:N', title='SBOM Tool'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.Color('upload_method:N', title='Upload Method'),
    tooltip=['detected_tool', 'upload_method', 'count']
).properties(title='SBOM Tool vs. Upload Method', height=300)
upload_chart.display()

## 12. Summary & Recommendations

Key findings and actionable recommendations.

In [ ]:
from rich.panel import Panel
from rich.markdown import Markdown

summary = f"""
# 🎯 SBOM Detection Strategy: Summary

## Key Findings

1. **Initial State**: Only 8 SBOM artifacts found in releases
2. **CI Generation**: {len(sbom_workflows['nameWithOwner'].unique())} repos have workflows mentioning SBOM keywords
3. **The Gap**: Many repos generate SBOMs in CI but don't publish them
4. **Popular Tools**: Syft, Trivy, and cdxgen are most common

## Recommendations

### For CNCF Projects
- ✅ Encourage publishing SBOMs as release artifacts
- ✅ Standardize SBOM naming conventions (e.g., `sbom.spdx.json`, `sbom.cdx.json`)
- ✅ Document SBOM generation in SECURITY.md or README.md
- ✅ Consider using GitHub Attestations API for SBOM storage

### For Analysis Improvement
- 🔧 Parse workflow YAML to extract structured step information
- 🔧 Check GitHub Actions artifact storage (not just releases)
- 🔧 Query GitHub Attestations API for SBOM attestations
- 🔧 Cross-reference with package registries (npm, PyPI, etc.)

### Next Steps
1. Export the gap analysis and share with CNCF projects
2. Enhance detection to find SBOMs in CI artifacts (not just releases)
3. Build a "SBOM maturity model" for projects
4. Create automated PR suggestions for SBOM publication
"""

console.print(Panel(Markdown(summary), title="Analysis Complete", border_style="green"))

## Cleanup

In [ ]:
# Close database connection
conn.close()
print("✅ Database connection closed.")